In [1]:
import sys
import os
sys.path.append("../Calculator_V2/")
from AngularPlot import AngularPlot
from multiprocessing import Pool
from PDF import PDF

In [2]:
Wavelengths=1
s_polarised_Intensity=2
p_polarised_Intensity=3

In [3]:
title="ang_el_OSRAM"
name="ang_el_OSRAM"
fileList=[["OSRAM_sample_5_3V_180muA.txt"],
         ]
labelList=["OSRAM Sample 15097-5", 
          ]
fitList=[
         ([140,220],[100,260],AngularPlot.cosine,(-1750,),(120,1750),None,{"maxfev":10000}),
         ]
desiredPlots=[
    {
        "xCol":Wavelengths,
        "yCol":s_polarised_Intensity,
        "yCol2":p_polarised_Intensity,
    }
]

In [4]:
figscale=0.6
present={
        "fig_width_pt":665.79451*figscale,
        "scaleX":(665.79451*figscale)/424.75906,
        "fixedFigWidth":True,
        "HWratio":0.82,
        "titleBool":False,
        }
inputParameters={
                    "labels":labelList,
                    "titleBool":False,
                    "customFontsize":[12,12,8,10,10],
                    "wavelength":600,
                    "fitList":fitList,
                    "doNotFit":True,
                    "showFitInLegend":False,
                    "errors":False
                }
inputParameters.update(present)
inputParametersForScaled=inputParameters.copy()
inputParametersForScaled.update({
                    "scaleX":0.48,
                    "customFontsize":[10,10,4,6,6],
                    "ax2Labels":False,
                    "titleBool":False
                    })
scaled=False

In [5]:
#optional
customLims=True # better pick False running first Time

xAxisLims=[None,None,None,None]
yAxisLims=[None,None,None,None]

In [6]:
def initPlot(xCol=1, showColTup=(2,3), customInputParameters=None):
    if customInputParameters is not None:
        inputParameters.update(customInputParameters)
        inputParametersForScaled.update(customInputParameters)
    scPlot=None
    if customLims:
        plot=AngularPlot(name,
                               fileList,
                               xCol=xCol,
                               showColTup=showColTup,
                               xLimOrig=xAxisLims[showColTup[0]],
                               showColAxLim=yAxisLims, 
                               **inputParameters)
        if scaled:
            scPlot=AngularPlot(name,
                                 fileList,
                                 xCol=xCol,
                                 showColTup=showColTup,
                                 xLimOrig=xAxisLims[showColTup[0]],
                                 showColAxLim=yAxisLims, 
                                 **inputParametersForScaled)
    else:
        plot=AngularPlot(name,
                               fileList,
                               xCol=xCol,
                               showColTup=showColTup,
                               **inputParameters)
        if scaled:
            scPlot=AngularPlot(name,
                                 fileList,
                                 xCol=xCol,
                                 showColTup=showColTup,
                                 **inputParametersForScaled)
    if scPlot is None:
        return [plot]
    return [plot,scPlot]

In [7]:
def buildPlotList(desiredPlot):
    try:
        yCol2=desiredPlot["yCol2"]
    except KeyError:
        yCol2=0
    try:
        cusPara=desiredPlot["custom"]
    except KeyError:
        cusPara=None
    if desiredPlot["yCol"]==0 or desiredPlot["xCol"]==0:
        raise
    return initPlot(xCol=desiredPlot["xCol"], showColTup=(desiredPlot["yCol"],yCol2), customInputParameters=cusPara)

In [8]:
def processPlotPair(plotpair):
    return [plot.doPlot() for plot in plotpair]

In [9]:
#multithreaded
pool = Pool(os.cpu_count())
plotList=pool.map(buildPlotList,desiredPlots)
multiOutput=pool.map(processPlotPair,plotList)
pool.close()

In [10]:
plots=[[plotOutput[0] for plotOutput in plotPair] for plotPair in multiOutput]
files=[[plotOutput[1] for plotOutput in plotPair] for plotPair in multiOutput]

In [11]:
PDF(files[0][0], size=(700,700*3/4)) #ang